In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
import torchvision.transforms.functional as t
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import time
import copy
import argparse
import os
from torch.autograd import variable 
from PIL import Image
import numpy as np
import json
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import scipy.misc


cuda:0


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


def initialize_model(model_name, num_classes, feature_extract):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained="imagenet")
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained="imagenet")
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained="imagenet")
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained="imagenet")
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet201(pretrained="imagenet")
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained="imagenet")
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    #chpt = torch.load(checkpoint_path)
    #model_ft.class_to_idx = chpt['class_to_idx']
    #model_ft.load_state_dict(chpt['state_dict'])

    return model_ft


mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
scale = 360
input_shape = 224
learning_rate = 0.0005
data_dir = "data_Task3/"
model_name = "densnet"
num_classes = 7
batch_size = 64

feature_extract = True


In [3]:
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
scale = 360
data_transforms = \
     transforms.Compose([
    transforms.Resize(scale),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=90),transforms.ToTensor(),
    transforms.Normalize(mean, std)])
df = pd.read_csv('gt.csv')

In [4]:
gt = pd.read_csv('data_task3/gt.csv')


In [5]:
from matplotlib import cm
gt_path = 'data_task3/gt.csv',
image_dir ='data_task3/image_train'

from PIL import Image
gt = pd.read_csv('data_task3/gt.csv')
gt.head()
image_dir ='data_task3/image_train'
img_name = os.path.join(image_dir,(df.iloc[2, 0]+'.jpg'))
image = Image.open(img_name)
img_name = os.path.join(image_dir,(
                                gt.iloc[3, 0]+'.jpg'))



gt =gt.iloc[2, 1:].values.astype(float)
gt = np.asarray([gt])
print(gt.shape)    
gt= torch.from_numpy(gt)
        
        #print(self.transform)
#image = Image.fromarray('image')
image = Image.open(img_name)
image =np.asarray(image)
image = Image.fromarray(image)

image = data_transforms(image)
image[2]
#imgplot = plt.imshow(image)

(1, 7)


tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

In [6]:
class ImageDataset():


    def __init__(self, gt ,image_dir , transform=data_transforms):
 
        self.gt = pd.read_csv(gt)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.gt)

    def __getitem__(self, idx):
        if type(idx) == torch.Tensor:
            idx = idx.item()
        img_name = os.path.join(self.image_dir,(
                                self.gt.iloc[idx, 0]+'.jpg'))
        image = Image.open(img_name)
        gt = self.gt.iloc[idx, 1:].values.astype(float)
        gt = np.asarray([gt])
        gt= torch.from_numpy(gt)
        
        #print(self.transform)
        image =np.asarray(image)
        image = Image.fromarray(image,  mode = 'RGB')
        image = self.transform(image)
        #sample = {'image': image, 'gt':gt}
        return  image, gt
       
       # return sample


In [7]:
def _make_dataloaders(dataset,  batch_size):
   
    dataset_size = len(dataset)
   
    valid_size = int(0.1*(dataset_size))
    train_size = dataset_size-valid_size
    
    train_dataset, valid_dataset =torch.utils.data.random_split(dataset, (train_size ,valid_size))
    train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
  
    valid = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)
    
    return train, valid



In [8]:
image_datasets  = ImageDataset( gt = 'data_task3/gt.csv', image_dir ='data_task3/image_train')
#print(len(image_datasets))
train, valid = _make_dataloaders(image_datasets, 32)
dataloaders = {'train': train, 'valid': valid}


In [9]:
def cross_entropy_one_hot(input, target):
    _, labels = target.max(dim=1)
    return nn.CrossEntropyLoss()(input, labels)

In [14]:

def train_model(model, dataloaders, learning_rate, num_epochs, criterion, is_inception=False):
    since = time.time()

    val_acc_history = []
    model.cuda()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and valididation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
              
                inputs = inputs.to(device)
                labels = labels.to(device=device, dtype=torch.int64)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs.type(torch.cuda.FloatTensor), labels.type(torch.cuda.LongTensor))
                        loss2 = criterion(aux_outputs.type(torch.cuda.FloatTensor), labels.type(torch.cuda.LongTensor))
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        print('label size = {}'.format(labels.shape))
                        labels = torch.argmax(labels, dim = 2)
                        #target =torch.argmax(labels, dim = 1)
                        
                        print('argmax(label) size = {}'.format(labels.shape))
                    
                        print('output shape = {}'.format(outputs.shape))
                        loss = criterion(outputs, labels )

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [15]:


num_epochs=25

learning_rate =0.001

input_shape = 224

model_name = "inception"
num_classes = 7
batch_size = 32

feature_extract = True




In [16]:
criterion = nn.CrossEntropyLoss()
model_ft = initialize_model('densenet', num_classes = 7, feature_extract = True)
optimizer = torch.optim.Adam(model_ft.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=True)
scheduler =  lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
image_datasets  = ImageDataset( gt = 'data_task3/gt.csv', image_dir ='data_task3/image_train')
#print(len(image_datasets))
train, valid = _make_dataloaders(image_datasets, 32)
dataloaders = {'train': train, 'valid': valid}
print(dataloaders['train'])

In [17]:
model_ft = train_model(model_ft, dataloaders, learning_rate, num_epochs, criterion, is_inception=False)

Epoch 0/24
----------
label size = torch.Size([32, 1, 7])
argmax(label) size = torch.Size([32, 1])
output shape = torch.Size([32, 7])


RuntimeError: multi-target not supported at c:\a\w\1\s\tmp_conda_3.6_090826\conda\conda-bld\pytorch_1550394668685\work\aten\src\thcunn\generic/ClassNLLCriterion.cu:15